In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob
import librosa
import pandas as pd

import torchaudio
from speechbrain.pretrained.interfaces import foreign_class

torchvision is not available - cannot save figures


In [2]:
classifier = foreign_class(source="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier", savedir="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", run_opts={"device":"cuda"})
accents = ["us", "england", "australia", "indian", "canada", "bermuda", "scotland", "african", "ireland", "newzealand", "wales", "malaysia", "philippines", "singapore", "hongkong", "southatlandtic",]

tempfile = "temp.wav"
def run_classification(path):
    audio, _ = librosa.load(path, sr=fs)
    torchaudio.save(tempfile, torch.tensor(audio).unsqueeze(0), fs)
    signal, org_sr = torchaudio.load(tempfile)
    signal = torchaudio.functional.resample(signal, orig_freq=org_sr, new_freq=fs)
    outputs =  classifier.encode_batch(signal)
    embeddings =  outputs[0].detach().cpu().numpy()
    outputs = classifier.mods.output_mlp(outputs)
    rawaccent = outputs[0].detach().cpu().numpy()
    probaccent = classifier.hparams.softmax(outputs).detach().cpu().numpy()[0]
    return embeddings, rawaccent, probaccent

def cosine_similarity(e1, e2): # from wespeaker, delete the normalizing part
    e1 = torch.tensor(e1)
    e2 = torch.tensor(e2)
    cosine_score = torch.dot(e1, e2) / (torch.norm(e1) * torch.norm(e2))
    cosine_score = cosine_score.item()
    return cosine_score

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [8]:
fs = 16000
accent_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"
native_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/English/wav/"
base_model_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"
model_paths = [
    "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_smaller/100000",
    "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_smaller/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
    
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller/100000",
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_smaller/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
]
base_dataset_dir = "/mntcephfs/lab_data/shoinoue/Dataset/"
dataset_paths = [
    "PD-AST/SLT/English",
    "PD-AST/SLT/Hindi",
    "PD-AST/SLT/Korean",
    "PD-AST/ASI/Hindi",
    "PD-AST/TNI/Hindi",
    "PD-AST/HKK/Korean",
    "PD-AST/YDCK/Korean",
    "CMU-ARCTIC/SLT",
    "L2-ARCTIC/ASI",
    "L2-ARCTIC/TNI",
    "L2-ARCTIC/HKK",
    "L2-ARCTIC/YDCK",
]

In [9]:
save_dir = f"evaluation/accent/"
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for ij in range(2):
    base_dir = [base_model_dir, base_dataset_dir][ij]
    dirnames = [model_paths, dataset_paths][ij]
    for dirname in dirnames:
        save_path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
        if os.path.exists(save_path):
            continue
        print(dirname)
        files = glob.glob(base_dir + f"{dirname}/wav/*.wav")
        files.sort()
        raws, probs, embs = [], [], []
        for path in tqdm(files):
            basename = os.path.basename(path)[:-4]
            if not(basename in testsamples):
                continue
            accent_path = f"./evaluation/accent_embeddings/Hindi/{basename}.npy"
            native_path = f"./evaluation/accent_embeddings/English/{basename}.npy"
            accent = np.load(accent_path, allow_pickle=True).item()
            native = np.load(native_path, allow_pickle=True).item()
            
            embeddings, rawaccent, probaccent = run_classification(path)
            a = cosine_similarity(embeddings, accent["embeddings"])
            b = cosine_similarity(embeddings, native["embeddings"])
            diff = a-b
            raws += [rawaccent[3]]
            probs += [probaccent[3]]
            embs += [diff]
            # if len(embs)>5:
            #     break

        np.save(save_path, [raws, probs, embs])

VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller/100000


100%|██████████| 100/100 [00:02<00:00, 33.60it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000


100%|██████████| 100/100 [00:02<00:00, 34.20it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000


...

0it [00:00, ?it/s]


VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000


100%|██████████| 5/5 [00:00<00:00, 32.35it/s]


VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000


100%|██████████| 5/5 [00:00<00:00, 34.71it/s]


In [10]:
def get_basic_info(key):
    key1 = key.split("/")[0]
    if "ARCTIC" in key1:
        data_type = "Ground Truth"
        speaker = key.split("/")[1]
        accent = spk2acc[speaker]
    elif "PD-AST"==key1:
        data_type = "SPAccenT"
        speaker, accent = key.split("/")[1:]
    else:
        data_type = "AC_"
        if "smaller" in key1:
            data_type += "small"
        elif "small" in key1:
            data_type += "normal"
        else:
            data_type += "large"
        speaker = "SLT"
        accent = mode
    return data_type, speaker, accent

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["classification score (before softmax)", "classification prob.", "AECS ratio"]:
    new_columns += [("scores", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)

spk2acc = {
    "SLT": "English",
    "ASI": "Hindi",
    "TNI": "Hindi",
    "HKK": "Korean",
    "YDCK": "Korean",
}

In [11]:
mode = "Hindi"
if mode=="Hindi":
    labels = [
        "CMU-ARCTIC/SLT",
        "L2-ARCTIC/ASI",
        "L2-ARCTIC/TNI",
        "PD-AST/SLT/English",
        "PD-AST/SLT/Hindi",
        "PD-AST/ASI/Hindi",
        "PD-AST/TNI/Hindi",
        # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_smaller/100000",
        # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
        # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_smaller/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    ]
elif mode=="Korean":
    labels = [
        "L2-ARCTIC/HKK",
        "L2-ARCTIC/YDCK",
        "PD-AST/SLT/English",
        "PD-AST/SLT/Korean",
        "PD-AST/HKK/Korean",
        "PD-AST/YDCK/Korean",
    ]

In [12]:
save_dir = f"evaluation/accent/"
basics = []
scores = []
for dirname in labels:
    data_type, speaker, accent = get_basic_info(dirname)
    path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
    raws, probs, embs = np.load(path)
    scores += [[raws.mean(), probs.mean(), embs.mean()]]
    basics += [[data_type, speaker, accent]]
    
array = np.concatenate([basics, scores], axis=1)
df = pd.DataFrame(array, columns=new_columns)
df.loc[:, "scores"] =  df.loc[:, "scores"].values.astype(float)
df

basics                                                 scores  \
       Data Type Speaker   Accent classification score (before softmax)   
0   Ground Truth     SLT  English                             -9.251676   
1   Ground Truth     ASI    Hindi                             13.304869   
2   Ground Truth     TNI    Hindi                              17.53969   
3       SPAccenT     SLT  English                              -9.04173   
4       SPAccenT     SLT    Hindi                             13.955082   
5       SPAccenT     ASI    Hindi                             11.895964   
6       SPAccenT     TNI    Hindi                             14.682575   
7       AC_small     SLT    Hindi                             16.172733   
8      AC_normal     SLT    Hindi                              16.50685   
9       AC_large     SLT    Hindi                             13.173969   
10      AC_small     SLT    Hindi                                   NaN   
11     AC_normal     SLT    Hindi                             17.808607   
12      AC_large     SLT    Hindi                             17.112174   

                                    
   classification prob. AECS ratio  
0                   0.0  -0.462143  
1              0.632042   0.223873  
2              0.883807   0.286671  
3                   0.0  -0.568643  
4              0.819328   0.568643  
5              0.353793   0.237296  
6              0.723657   0.277133  
7              0.897837   0.466295  
8              0.943469   0.487807  
9              0.835076   0.436191  
10                  NaN        NaN  
11             0.948258   0.525615  
12             0.906899   0.505283

---
---
---
---

In [82]:
import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [94]:
dirnames

['PD-AST/SLT/English',
 'PD-AST/SLT/Hindi',
 'PD-AST/SLT/Korean',
 'PD-AST/ASI/Hindi',
 'PD-AST/TNI/Hindi',
 'PD-AST/HKK/Korean',
 'PD-AST/YDCK/Korean',
 'CMU-ARCTIC/SLT',
 'L2-ARCTIC/ASI',
 'L2-ARCTIC/TNI',
 'L2-ARCTIC/HKK',
 'L2-ARCTIC/YDCK']

In [108]:
dirnames[3]

'PD-AST/ASI/Hindi'

In [119]:
save_dir = f"evaluation/accent/"
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for ij in range(1):
    base_dir = [base_dataset_dir][ij]
    dirnames = [dataset_paths][ij]
    for dirname in [dirnames[3]]:
        print(dirname)
        files = glob.glob(base_dir + f"{dirname}/wav/*.wav")
        files.sort()
        raws, probs, embs = [], [], []
        for path in tqdm(files):
            basename = os.path.basename(path)[:-4]
            if not(basename in testsamples):
                continue
            accent_path = f"./evaluation/accent_embeddings/Hindi/{basename}.npy"
            native_path = f"./evaluation/accent_embeddings/English/{basename}.npy"
            accent = np.load(accent_path, allow_pickle=True).item()
            native = np.load(native_path, allow_pickle=True).item()
            
            embeddings, rawaccent, probaccent = run_classification(path)
            a = cosine_similarity(embeddings, accent["embeddings"])
            b = cosine_similarity(embeddings, native["embeddings"])
            diff = a-b
            raws += [rawaccent[3]]
            probs += [probaccent[3]]
            print(probaccent[3])
            # if probaccent[3]<0.1:
                # break

PD-AST/ASI/Hindi


  3%|▎         | 3/100 [00:00<00:03, 25.17it/s]

0.00014754343
0.008029114
0.00015568953
0.030948617
0.016338497


  6%|▌         | 6/100 [00:00<00:03, 25.14it/s]

0.12190316


...

0.99905723
0.9700633
0.010576587
0.0039412044
0.25270325
0.99415106
0.9322106


 94%|█████████▍| 94/100 [00:02<00:00, 32.21it/s]

0.365449
0.09940741
2.4538922e-05
0.09791985
0.5217823
0.9882206
0.9999753


100%|██████████| 100/100 [00:03<00:00, 32.41it/s]

0.42606127
0.9980476
0.0002677649


In [117]:
rawaccent

array([ 17.33565  ,   3.4303904, -13.373028 ,   8.514436 ,  -7.551818 ,
        -5.6582313, -10.052798 , -16.763783 ,  -6.071382 , -16.28312  ,
       -10.090539 , -14.04483  , -11.300481 , -10.176998 ,  -7.3399386,
       -15.123959 ], dtype=float32)

In [118]:
dirname

'PD-AST/ASI/Hindi'

In [112]:
basename

'arctic_a0024'

In [116]:
play_audio(path, 16000)

In [88]:
path

'/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/wav/arctic_a0124.wav'

In [85]:
basename

'arctic_a0124'

In [ ]:
play